In [1]:
# Importa os módulos necessários.
import os  # Permite interagir com o sistema operacional, como acessar variáveis de ambiente.
import openai  # Biblioteca oficial da OpenAI para interagir com a API do ChatGPT.
import json  # Biblioteca para trabalhar com dados no formato JSON.
from dotenv import load_dotenv, find_dotenv  # Importa funções para carregar variáveis de ambiente de um arquivo .env.

# Carrega as variáveis de ambiente do arquivo .env.
_ = load_dotenv(find_dotenv())

# Obtém a chave da API da OpenAI a partir das variáveis de ambiente carregadas.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Cria um cliente da OpenAI para interagir com os modelos de linguagem.
client = openai.Client()


# Uso da API do ChatGPT com Funções e Ferramentas Externas

Este script demonstra como utilizar a API do ChatGPT da OpenAI para obter a temperatura atual de determinadas cidades, incorporando funções personalizadas e ferramentas externas. O código utiliza a biblioteca `openai` para interagir com a API e o pacote `dotenv` para carregar a chave da API de um arquivo `.env`.

## Requisitos
- Python 3.x
- Pacotes: `openai`, `dotenv`
- Um arquivo `.env` contendo sua chave da API da OpenAI (`OPENAI_API_KEY`)

## Descrição do Código

### 1. Importação das Bibliotecas
As bibliotecas necessárias são importadas: 
- `os` para acessar variáveis de ambiente.
- `openai` para interagir com a API do ChatGPT.
- `json` para manipular dados em formato JSON.
- `dotenv` para carregar variáveis de ambiente de um arquivo `.env`.

### 2. Carregamento das Variáveis de Ambiente
Utiliza `load_dotenv()` para carregar variáveis do arquivo `.env`, incluindo a chave da API da OpenAI.

### 3. Criação do Cliente OpenAI
Inicializa um cliente da OpenAI para facilitar as interações com os modelos de linguagem.

### 4. Função `obter_temperatura_atual`
Define uma função para simular a obtenção da temperatura atual em diferentes cidades. Retorna os dados no formato JSON.

### 5. Ferramentas para a API do ChatGPT
Define uma lista de ferramentas (neste caso, uma função) que a API do ChatGPT pode usar para obter dados externos. A ferramenta é configurada com os parâmetros necessários para realizar a consulta.

### 6. Interação com o ChatGPT
Prepara uma mensagem de entrada do usuário e faz uma chamada à API do ChatGPT para processar a solicitação. O código é capaz de identificar se há necessidade de utilizar a ferramenta definida e a chama conforme necessário.

### 7. Processamento das Respostas
Se o ChatGPT solicitar a execução de uma ferramenta, o código extrai a chamada da função, executa-a, e armazena a resposta para enviar uma nova mensagem ao modelo, obtendo assim a resposta final.

### 8. Saída da Resposta
Exibe a resposta final do modelo de linguagem.

In [2]:
# Função que obtém a temperatura atual de uma cidade.
# 'local' é o nome da cidade a ser consultada e 'unidade' é a unidade de temperatura (padrão é "celsius").
def obter_temperatura_atual(local, unidade="celsius"):
    # Verifica se a cidade é "São Paulo" (não faz distinção entre maiúsculas e minúsculas).
    if "são paulo" in local.lower():
        # Retorna um JSON contendo as informações da temperatura de São Paulo.
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
        )
    # Verifica se a cidade é "Porto Alegre".
    elif "porto alegre" in local.lower():
        # Retorna um JSON contendo as informações da temperatura de Porto Alegre.
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
        )
    # Verifica se a cidade é "Rio de Janeiro".
    elif "rio de janeiro" in local.lower():
        # Retorna um JSON contendo as informações da temperatura do Rio de Janeiro.
        return json.dumps(
            {"local": "Rio de Janeiro", "temperatura": "35", "unidade": unidade}
        )
    # Se a cidade não for nenhuma das mencionadas acima, retorna um JSON com temperatura desconhecida.
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
        )

# Lista de ferramentas (neste caso, uma função) que a API do ChatGPT pode usar para obter dados externos.
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    # Define o parâmetro 'local' que é o nome da cidade.
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    # Define o parâmetro 'unidade' para especificar a unidade de temperatura.
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                # Torna 'local' um parâmetro obrigatório.
                "required": ["local"],
            },
        },
    }
]

# Dicionário que mapeia os nomes das funções disponíveis para os objetos de função correspondentes.
funcoes_disponiveis = {
    "obter_temperatura_atual": obter_temperatura_atual,
}

# Lista de mensagens que contém a interação inicial do usuário.
mensagens = [
    {
        "role": "user", 
        "content": "Qual é a temperatura em São Paulo e Porto Alegre?"
    }
]

# Realiza uma chamada para a API do ChatGPT, especificando o modelo e fornecendo as mensagens e ferramentas.
resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools,
    tool_choice="auto",  # Permite que o modelo escolha a ferramenta a ser utilizada automaticamente.
)

# Extrai a primeira resposta gerada pelo modelo da lista de escolhas.
mensagem_resp = resposta.choices[0].message

# Extrai as chamadas de ferramenta (tool_calls) da resposta do modelo, se houver.
tool_calls = mensagem_resp.tool_calls

# Se houver chamadas de ferramenta...
if tool_calls:
    # Adiciona a resposta do modelo à lista de mensagens.
    mensagens.append(mensagem_resp)
    # Itera por todas as chamadas de ferramenta.
    for tool_call in tool_calls:
        # Obtém o nome da função a ser chamada.
        function_name = tool_call.function.name
        # Obtém a função correspondente do dicionário de funções disponíveis.
        function_to_call = funcoes_disponiveis[function_name]
        # Extrai os argumentos da chamada da função.
        function_args = json.loads(tool_call.function.arguments)
        # Chama a função com os argumentos fornecidos (local e unidade).
        function_response = function_to_call(
            local=function_args.get("local"),
            unidade=function_args.get("unidade"),
        )
        # Adiciona a resposta da ferramenta à lista de mensagens.
        mensagens.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
    # Faz uma segunda chamada à API do ChatGPT com as mensagens atualizadas, incluindo a resposta da ferramenta.
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens,
    )

# Extrai e imprime o conteúdo da segunda resposta.
mensagem_resp = segunda_resposta.choices[0].message
print(mensagem_resp.content)


A temperatura atual em São Paulo é de 32°C e em Porto Alegre é de 25°C.
